<h1> SEMRush & GSC Script - Top Opportunities</h1>

In [ ]:
import pandas
from api import *
from datetime import datetime
from bs4 import BeautifulSoup
import requests
import random

<h2> Input Criteria - Form Inputs On-page </h2>

In [ ]:
# FORM INPUT FIELDS

domain_lookup = 'https://www.inseev.com/'

# GSC Data Input 
gsc_property = 'https://www.inseev.com/'
startdate="2020-12-20" # Make 30 day max
enddate="2020-12-30" # Make 30 day max

# SEMRush Data
## Brand Lookup ("brandNgram1|brandNgram2|brandNgram3....")
nonbrand_semrush = "inseev|insev|insv"
semrush_csv= "inseev.com-organic.Positions-us-20210122-2021-01-23T21_50_40Z.csv"
nlargest_nbKeywords = 15
top_pages_sort = 'Search_Volume_semrush' # Dropdown filter (CRITERIA BELOW)
top_queries_sort ='Traffic_semrush' # Dropdown filter (CRITERIA BELOW)


###### FORM FILL FIELDS / INSTRUCTION #######

# GSC Data Input 
## gsc_property - Input Type: Text Box (Frontend Header: 'GSC Property')
## startdate - Input Type: Date Selector (Frontend Header: 'Start Date') - MAX 30 DAY RANGE
## enddate - Input Type: Date Selector (Frontend Header: 'End Date') - MAX 30 DAY RANGE

# SEMRush Data
## nonbrand_semrush - Input Type: Text Box (Frontend Header: 'Brand Exclusions'); Subtext showing "e.g., brandNgram1|brandNgram2|brandNgram3...."
## semrush_csv - Input Type: CSV Input (Frontend Header: 'CSV Import')
## nlargest_nbKeywords - Input Type: Integer (1 to 15)  (Frontend Header: 'Number of Returned Keywords')
## top_pages_sort - Input Type: Dropdown Menu (Frontend Header: 'Select Sort by (Top Pages)')

### Option #1: 'Search_Volume_semrush' (Frontend Text: "Total Search Volume (Top 20 KWs)")
### Option #2: 'Keyword_semrush' (Frontend Text: "Total Keywords (Top 20 KWs)")

## top_queries_sort - Input Type: Dropdown Menu (Frontend Header: 'Select Sort by (Top Keywords)')
### Option #1: 'Search_Volume_semrush' (Frontend Text: "Search Volume")
### Option #2: 'Traffic_semrush' (Frontend Text: "Estimated Traffic")



<h2>PART 1: Get + Clean GSC Data </h2>

In [ ]:
domain_name = domain_lookup.split("www.")[-1].split("//")[-1].split(".")[0]
date = datetime.today().strftime('%Y-%m-%d')

# Options: 'date,' 'device,' 'page,' , 'query' and "country"
dimensions=['page','query','device','date']

gsc_df = gscservice.get_site_data(
    gsc_property,
    startdate=startdate,
    enddate=enddate,
    dimensions=dimensions,
     output_fn="{}_{}_{}_by_{}.csv".format(domain_name, startdate.replace("-",""), enddate.replace("-",""), '_'.join(dimensions))
)


# Filter to only non-brand
kw_filter = ~gsc_df["query"].str.contains("{}".format(nonbrand_semrush), case = False, regex=True)
gsc_df = gsc_df[kw_filter]


# Insert domain & datasetID
gsc_df.insert(loc = 2, column = "domain", value = domain_lookup)
datasetID = "{}_{}_{}_by_{}".format(domain_name, startdate.replace("-",""), enddate.replace("-",""), '_'.join(dimensions))
gsc_df.insert(loc = 0, column = "gsc_datasetID", value = datasetID)

gsc_df.head(3) 

In [ ]:
select_cols = ["gsc_datasetID","domain", "page", "query", "clicks", "impressions","ctr","position"]
gsc_df = gsc_df[select_cols]
grouped_gsc = gsc_df.groupby(["gsc_datasetID","domain", "page","query"])

gsc_all_data = gsc_df.groupby(["gsc_datasetID","domain","page","query"])
gsc_all_data = gsc_all_data.agg({"clicks": "sum",
           "impressions": "sum",
           "ctr" : "mean",
           "position":["size","max","min", "mean"]})

gsc_all_data = gsc_all_data.reset_index()

<h2> Part 2: Get + Clean SEMRush Data</h2>

In [ ]:
# Read CSV + update column names to not contain spaces

semrush_df = pandas.read_csv("{}".format(semrush_csv))
semrush_df.columns = [column_name.replace(" ","_") for column_name in semrush_df.columns]

# Filter to only non-brand queries
filter_kw_type = ~semrush_df["Keyword"].str.contains("{}".format(nonbrand_semrush), case = False, regex=True)
semrush_df = semrush_df[filter_kw_type]
semrush_df.head(3)

In [ ]:
def position_range(position):
    if position < 11:
        return "1 to 10"
    elif position >= 11 and position <= 20:
        return "11 to 20"
    elif position >= 21 and position <= 30:
        return "11 to 20"
    else:
        return "31+"
    
def top20(position):
    if position < 21:
        return "TRUE"
    else:
        return "FALSE"
    
def traffic_exists(traffic):
    if traffic > 0:
        return "TRUE"
    else:
        return "FALSE"
    
semrush_df["Traffic_Cost"] = semrush_df["Traffic_Cost"].fillna(0).astype("int")


select_cols = ["Keyword", "Position", "Search_Volume", "CPC", "URL", "Traffic", "Traffic_Cost", "Timestamp", "SERP_Features_by_Keyword"]
semrush_df = semrush_df[select_cols]
    
semrush_df.insert(loc = 2, column = "Position_Range", value = semrush_df["Position"].apply(position_range))
semrush_df.insert(loc = 3, column = "Top20", value = semrush_df["Position"].apply(top20))
semrush_df.insert(loc = 6, column = "Domain", value = domain_lookup)
semrush_df.insert(loc = 9, column = "Traffic_Exists", value = semrush_df["Traffic"].apply(traffic_exists))
semrush_df.insert(loc = 9, column = "Traffic_Rank", value = semrush_df["Traffic"].rank(ascending = False).astype(int))

semrush_df.sort_values("Traffic", ascending = False, inplace = True)
semrush_df.columns = [str(col) + '_semrush' for col in semrush_df.columns]
semrush_df.head(4)

<h2> Part 3: Scrape Top Pages & Merge Data </h2>

In [ ]:
### [BELOW AGGREGATES PAGES FOR SCRAPER]
# Select only areas where "Top20_semrush" == True 
top20filter = semrush_df["Top20_semrush"].str.contains("TRUE", case = False, regex=True)
top20queries = semrush_df[top20filter]
top20queries = top20queries.sort_values(by = "Traffic_semrush", ascending=False)
domainFilter = top20queries["Domain_semrush"].str.contains("{}".format(domain_lookup), case = False, regex=True)
cleanDomain_top20Queries_df = top20queries[domainFilter]

# groupby #1 - page-level
semrush_url_group = cleanDomain_top20Queries_df.groupby(["Domain_semrush","URL_semrush"])
semrush_url_group = semrush_url_group.agg({"Traffic_semrush": "sum",
           "Position_semrush":"mean", 
            "Search_Volume_semrush": "sum",
            "Keyword_semrush":"count"})

# reset index and groupby URL_semrush in order to setup for "for loop"
semrush_url_group = semrush_url_group.reset_index() 
select_cols = ["Domain_semrush","URL_semrush", "Traffic_semrush", "Position_semrush", "Search_Volume_semrush", "Keyword_semrush"]
semrush_df_columns = semrush_url_group[select_cols]
semrush_df_columns  = pandas.DataFrame(columns = semrush_df_columns.columns)

# groupby #2 - "Domain_semrush" - only 1 item so we'll get top pages w/ nlargest

semrush_url_group = semrush_url_group.groupby("Domain_semrush")

for URL_semrush, data in semrush_url_group:
    highest_keywords_in_group = data.nlargest(25,"{}".format(top_pages_sort))
    semrush_df_columns = semrush_df_columns.append(highest_keywords_in_group)
    
top_page_kpis = semrush_df_columns
    
### [BELOW IS THE SCRAPER]
# Dictionaries updated w/ looped data
titleTag= {}
metaDescription = {}
h1Tag = {}

semrush_df_columns = (semrush_df_columns["URL_semrush"]).unique()

def GET_UA():
    uastrings = ["Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36",\
                "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1500.72 Safari/537.36",\
                "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10) AppleWebKit/600.1.25 (KHTML, like Gecko) Version/8.0 Safari/600.1.25",\
                "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:33.0) Gecko/20100101 Firefox/33.0",\
                "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36",\
                "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36",\
                "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/600.1.17 (KHTML, like Gecko) Version/7.1 Safari/537.85.10",\
                "Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko",\
                "Mozilla/5.0 (Windows NT 6.3; WOW64; rv:33.0) Gecko/20100101 Firefox/33.0",\
                "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.104 Safari/537.36"\
                ]
 
    return random.choice(uastrings)


for url in semrush_df_columns:
    USER_AGENT = GET_UA()
    headers = {'user-agent': USER_AGENT}
    resp= requests.get(url, headers=headers)

    #Parse the HTML with Beautiful soup
    if resp.status_code == 200:
        bs = BeautifulSoup(resp.content, "html.parser")
        
    
    #to get the title tags
    title= bs.find('title')
    titleTag.update([(url,title)])
    
    #to get the h1 tags
#     h1 = bs.body.find('h1')
#     h1Tag.update([(url,h1)])


    #this is to get the meta descriptions
    for meta in bs.find_all('meta'):
        if meta.get('name')== 'description':
            metaDescription.update([(meta.get('content'), url)])

# Can use any of the Dictionaries stated at the start of the function

titleTag_df = pandas.DataFrame(list(titleTag.items()), columns=["Page_Scrape","Title_Scrape"])
# h1Tag_df = pandas.DataFrame(list(h1Tag.items()), columns=["Page_Scrape","H1Tag_Scrape"])
# seo_data_scrape_topPages = titleTag_df.merge(h1Tag_df, how="inner", left_on= ["Page_Scrape"], right_on=["Page_Scrape"])
topPages_Scrape_merge = titleTag_df.merge(top_page_kpis, how="inner", left_on= ["Page_Scrape"], right_on=["URL_semrush"])

In [ ]:
# Perform initial groupby to get KPIs we want at the page-level
top20queries_2 = semrush_df[top20filter]
semrush_topQuery_df = top20queries_2.groupby(["Domain_semrush","URL_semrush","Keyword_semrush"])
semrush_topQuery_df = semrush_topQuery_df.agg({"Traffic_semrush": "sum",
           "Position_semrush":"mean", 
            "Search_Volume_semrush": "sum"})

# reset index and groupby URL_semrush in order to setup for "for loop"
semrush_topQuery_df = semrush_topQuery_df.reset_index() 

select_cols_pt2 = ["Domain_semrush","URL_semrush", "Keyword_semrush", "Traffic_semrush", "Position_semrush", "Search_Volume_semrush"]
semrush_df_columns_pt2 = semrush_topQuery_df[select_cols]
semrush_df_columns_pt2  = pandas.DataFrame(columns = semrush_df_columns_pt2.columns)

# For loop to get top XX queries
semrush_topQuery_df = semrush_topQuery_df.groupby("URL_semrush")
semrush_topQuery_df

for URL_semrush, data in semrush_topQuery_df:
    highest_keywords_in_group = data.nlargest(int("{}".format(nlargest_nbKeywords)),"{}".format(top_queries_sort))
    semrush_df_columns_pt2 = semrush_df_columns_pt2.append(highest_keywords_in_group)
    
merge_top20 = semrush_df_columns_pt2.merge(topPages_Scrape_merge, how="left", left_on= ["URL_semrush", "Domain_semrush"], right_on=["URL_semrush", "Domain_semrush"])
merge_top20_gsc = merge_top20.merge(gsc_all_data, how="left",  left_on= ["URL_semrush", "Keyword_semrush_x"], right_on=[('page',''), ('query','')])

merge_top20_gsc = merge_top20_gsc.rename(mapper={"Keyword_semrush_y":"#_keywords_top20",
                                        'Traffic_semrush_y':'est_traffic_top20',
                                         "Search_Volume_semrush_y": "total_volume_top20",
                                         "Traffic_semrush_x":"total_keywords_top20", 
                                         "Keyword_semrush_x":"top_keywords", 
                                         "query": "matching_gsc_query",
                                         "Search_Volume_semrush_x":"top_keyword_volume", 
                                         "Position_semrush_x":"top_keyword_position", 
                                         "Traffic_semrush_x":"top_keyword_traffic",
                                        ("clicks","sum"):"clicks_gsc",
                                        ("impressions","sum"):"impressions_gsc",
#                                         "H1Tag_Scrape": "H1_tag",
                                        "Title_Scrape": "Title_tag",
                                         "URL_semrush": "URL_semrush"}, axis="columns")


select_cols_final = ["URL_semrush","Title_tag", "#_keywords_top20", "est_traffic_top20", "total_volume_top20", "top_keywords","top_keyword_volume","top_keyword_position", "top_keyword_traffic","clicks_gsc","impressions_gsc"]
merge_top20_gsc = merge_top20_gsc[select_cols_final]
naFilter = merge_top20_gsc["#_keywords_top20"].notna()
merge_top20_final = merge_top20_gsc[naFilter]

merge_top20_final.to_csv("semrush-opportunity-analysis_{}-{}.csv".format(domain_name,date),  index=True)
merge_top20_final.head(5)